In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
from tokenizers import ByteLevelBPETokenizer
from pathlib import Path

In [3]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/filtered-dumping-academia.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/filtered-dumping-wiki.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/dumping-iium.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/dumping-news.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/dumping-parliament.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/dumping-pdf.txt
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/dumping-watpadd.txt

In [4]:
tokenizer = ByteLevelBPETokenizer(lowercase=False)

In [5]:
from glob import glob

files = glob('filtered-dumping-*.txt')
files.extend(glob('dumping-*.txt'))
files = [f for f in files if 'twitter' not in f and 'instagram' not in f]
files.extend(['bahasa-asr-train.txt', 'bahasa-asr-test.txt'])
files = sorted(files)
files

['bahasa-asr-test.txt',
 'bahasa-asr-train.txt',
 'dumping-iium.txt',
 'dumping-news.txt',
 'dumping-parliament.txt',
 'dumping-pdf.txt',
 'dumping-watpadd.txt',
 'filtered-dumping-academia.txt',
 'filtered-dumping-wiki.txt']

In [6]:
tokenizer.train(files=files, vocab_size=32000, min_frequency=2,
                show_progress=True,
                special_tokens = ['<s>', '<pad>', '</s>', '<|endoftext|>'],)

In [7]:
!mkdir gpt2_bpe

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
mkdir: cannot create directory ‘gpt2_bpe’: File exists


In [8]:
tokenizer.save_model('gpt2_bpe')

['gpt2_bpe/vocab.json', 'gpt2_bpe/merges.txt']

In [9]:
import torch
import numpy as np
import transformers
from transformers import GPT2Tokenizer

In [10]:
tokenizer2 = GPT2Tokenizer.from_pretrained('./gpt2_bpe', do_lower_case = False)
tokenizer2.model_max_length = 1024
tokenizer2.save_pretrained('malay-cased-gpt2')

('malay-cased-gpt2/tokenizer_config.json',
 'malay-cased-gpt2/special_tokens_map.json',
 'malay-cased-gpt2/vocab.json',
 'malay-cased-gpt2/merges.txt',
 'malay-cased-gpt2/added_tokens.json')

In [11]:
GPT2Tokenizer.from_pretrained('malay-cased-gpt2')

PreTrainedTokenizer(name_or_path='malay-cased-gpt2', vocab_size=32000, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True)})

In [12]:
tokenizer2.encode('saya')

[3795]

In [13]:
tokenizer2.encode(tokenizer2.bos_token)

[3]

In [14]:
!head -n 10000 filtered-dumping-wiki.txt > sample-wiki.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
' '.join(files)

'bahasa-asr-test.txt bahasa-asr-train.txt dumping-iium.txt dumping-news.txt dumping-parliament.txt dumping-pdf.txt dumping-watpadd.txt filtered-dumping-academia.txt filtered-dumping-wiki.txt'

In [16]:
cat {' '.join(files)} >> train-v2.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
!wc -w train-v2.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
930544970 train-v2.txt


In [18]:
!head -n 10 train-v2.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
inilah dunia kecil
itu tadi menarik ya
pelayan restoran ini kelihatannya sibuk
apakah anda pernah makan serangga
aku pernah ingin menjadi seorang astronom fisika
saya sedang mencari dompet hitam seperti ini
apakah anda percaya pada tuhan
semuanya sedang menunggu guru di kelas
anak-anak kecil suka dengan buku tentang dinosaurus dan monster
ini yang dilakukan mary untuk hidup


In [19]:
!tail -n 10 train-v2.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Ciamannacce ialah komun di jabatan Corse-du-Sud di kepulauan Corsica, Perancis.


Coggia ialah komun di jabatan Corse-du-Sud di kepulauan Corsica, Perancis.


Cognocoli-Monticchi ialah komun di jabatan Corse-du-Sud di kepulauan Corsica, Perancis.


Conca ialah komun di jabatan Corse-du-Sud di kepulauan Corsica, Perancis.
